In [ ]:
# Connect to IN-CORE Services
from pyincore import IncoreClient

client = IncoreClient()

In [ ]:
# testing datasets 

tornado_hazard_id = "5d07cbeab9219c065b080930"

joplin_bldg_inv_id = "5dbc8478b9219c06dd242c0d"

eq_hazard_id = "5b902cb273c3371e1236b36b"

shelby_hopital_inv_id = "5a284f0bc7d30d13bc081a28"

shelby_road_id = "5a284f2bc7d30d13bc081eb6"

In [ ]:
# Creating a Dataset object with ID and Data Service

from pyincore.dataservice import DataService
from pyincore.hazardservice import HazardService
from pyincore import Dataset
from pyincore_viz.geoutil import GeoUtil as viz

In [ ]:
eq_metadata  = HazardService(client).get_earthquake_hazard_metadata(eq_hazard_id)
eq_dataset_id = eq_metadata['rasterDataset']['datasetId']

eq_dataset = Dataset.from_data_service(eq_dataset_id, DataService(client))

In [ ]:
viz.plot_earthquake(eq_hazard_id, client)

In [ ]:
sh_bldg_inv = Dataset.from_data_service(shelby_hopital_inv_id, DataService(client))

In [ ]:
sh_road = Dataset.from_data_service(shelby_road_id, DataService(client))

In [ ]:
viz.plot_map(sh_bldg_inv, column="struct_typ", category=False, basemap=True)

In [ ]:
viz.get_wms_map([sh_bldg_inv, sh_road],zoom_level=10)

In [ ]:
viz.get_gdf_map([sh_bldg_inv, sh_road],zoom_level=10)

In [ ]:
viz.get_gdf_wms_map([sh_bldg_inv], [sh_road], zoom_level=10)

In [ ]:
viz.plot_tornado(tornado_hazard_id, client, basemap=False)

In [ ]:
tornado_dataset_id = HazardService(client).get_tornado_hazard_metadata(tornado_hazard_id)['datasetId']
tornado_dataset = Dataset.from_data_service(tornado_dataset_id, DataService(client))

In [ ]:
viz.get_gdf_map([tornado_dataset], zoom_level=11)

In [ ]:
joplin_bldg_inv = Dataset.from_data_service(joplin_bldg_inv_id, DataService(client))

In [ ]:
# using wms layer for joplin building inv. gdf will crash the browser
viz.get_gdf_wms_map([tornado_dataset], [joplin_bldg_inv], zoom_level=11)